### Setting up file paths

In [ ]:
# URL of the file you want to download
url = 'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_candidato_munzona/votacao_candidato_munzona_2022.zip'
# The local path where you want to save the file
workdir = './work/'
parquetdir = './parquet/'
destfile = 'votacao_candidato_munzona_2022.zip'

In [ ]:
source_csv = 'votacao_candidato_munzona_2022_BRASIL.csv'
from zipfile import ZipFile
with ZipFile(workdir+destfile, 'r') as zObject: 
  
    # Extracting all the members of the zip  
    # into a specific location. 
    zObject.extract(source_csv, workdir + 'extracted')
extracted_file = workdir +'extracted/'+source_csv

In [2]:
workdir = './work/'
parquetdir = './parquet/'
source_csv = 'votacao_candidato_munzona_2022_BRASIL.csv'
extracted_file = workdir +'extracted/'+source_csv

In [3]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Creating Spark session

In [5]:
from pyspark.sql import SparkSession

jdbc_driver_path = "postgresql-42.7.1.jar"
# Create a SparkSession
spark_session = SparkSession.builder.appName('spark').config("spark.driver.extraClassPath", jdbc_driver_path).getOrCreate()

### Reading file

In [5]:
df = spark_session.read.options(header="true", delimiter=";", encoding="ISO-8859-1").csv(extracted_file)

In [15]:
import pandas as pd
GZIP_INPUT_FILE = './data/votacao.parquet.gzip'


df = pd.read_parquet(GZIP_INPUT_FILE)

### Selecting relevant columns

In [8]:
# Defining relevant columns
relevant_columns=[
    "NR_TURNO",
    "DS_ELEICAO",
    "TP_ABRANGENCIA",
    "SG_UF",
    "NM_MUNICIPIO",
    "NR_ZONA",
    "DS_CARGO",
    "NR_CANDIDATO",
    "NM_CANDIDATO",
    "NM_URNA_CANDIDATO",
    "DS_SITUACAO_CANDIDATURA",
    "NR_PARTIDO",
    "SG_PARTIDO",
    "NM_PARTIDO",
    "NM_COLIGACAO",
    "DS_COMPOSICAO_COLIGACAO",
    "ST_VOTO_EM_TRANSITO",
    "QT_VOTOS_NOMINAIS",
    "NM_TIPO_DESTINACAO_VOTOS",
    "QT_VOTOS_NOMINAIS_VALIDOS",
    "DS_SIT_TOT_TURNO"
]

# Selecting relevant columns
selected_columns_df = df.select(relevant_columns)

NameError: name 'df' is not defined

### Printing first lines

In [16]:
df.show()

AttributeError: 'DataFrame' object has no attribute 'show'

### Writing to parquet file

In [12]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

def transform_spark_dataframe_into_star_schema(
    dataframe,
    colunas_fato = ["col1", "col2"],
    tabela_fato_nome = "tabela_fato",
    mapping_colunas_dimensao = {'dim1':["col3", "col4"], "dim2":["col5", "col6"]},
):

    colunas_fato_e_dimensao = colunas_fato + [col for cols in mapping_colunas_dimensao.values() for col in cols]
    dataframe = dataframe.select(*colunas_fato_e_dimensao)

    dimensions = []
    for dim, cols in mapping_colunas_dimensao.items():

        df_dimension = dataframe.select(*cols).distinct()
        sk_name = f"sk_{dim.replace('DIM_', '')}"
        # add unique id to dimension
        df_dimension = df_dimension.withColumn(sk_name, F.monotonically_increasing_id())

        dimensions.append( (dim, df_dimension) )



    # Substitui as colunas de dimensão pelo respectivo SK na tabela fato
    # ------------------------------------------------------------------
    for dim, df_dimension in dimensions:
        # join the dimension dataframe to the original dataframe
        dataframe = dataframe.join(
            df_dimension, 
            on=[
                dataframe[col] == df_dimension[col]
                for col in mapping_colunas_dimensao[dim]
            ],
            how="left"
        )

        # drop the original columns
        dataframe = dataframe.drop(*mapping_colunas_dimensao[dim])
    return [ (tabela_fato_nome, dataframe) ] + dimensions


In [17]:
import pandas as pd
import numpy as np

def transform_pandas_dataframe_into_star_schema(
    dataframe,
    colunas_fato = ["col1", "col2"],
    tabela_fato_nome = "tabela_fato",
    mapping_colunas_dimensao = {'dim1':["col3", "col4"], "dim2":["col5", "col6"]},
):

    colunas_fato_e_dimensao = colunas_fato + [col for cols in mapping_colunas_dimensao.values() for col in cols]
    dataframe = dataframe[colunas_fato_e_dimensao]

    dimensions = []
    for dim, cols in mapping_colunas_dimensao.items():

        df_dimension = dataframe[cols].drop_duplicates()
        sk_name = f"sk_{dim.replace('DIM_', '')}"
        # add unique id to dimension
        df_dimension[sk_name] = np.arange(len(df_dimension))

        dimensions.append( (dim, df_dimension) )

    # Substitui as colunas de dimensão pelo respectivo SK na tabela fato
    # ------------------------------------------------------------------
    for dim, df_dimension in dimensions:
        # join the dimension dataframe to the original dataframe
        dataframe = pd.merge(
            dataframe, 
            df_dimension, 
            on=cols,
            how="left"
        )

        # drop the original columns
        dataframe = dataframe.drop(columns=cols)
    return [ (tabela_fato_nome, dataframe) ] + dimensions

In [18]:
star_schema = transform_spark_dataframe_into_star_schema(
    df,
    colunas_fato=["QT_VOTOS_NOMINAIS_VALIDOS", "QT_VOTOS_NOMINAIS"],
    tabela_fato_nome="tabela_fato",
    mapping_colunas_dimensao={
        'dim_municipio': ["SG_UF", "NM_MUNICIPIO"],
        'dim_cargo': ["DS_CARGO"],
        'dim_ds_eleicao':["DS_ELEICAO"],
        'dim_partido':["SG_PARTIDO","NM_PARTIDO", "NR_PARTIDO"],
        'dim_candidato':["NM_CANDIDATO", "NR_CANDIDATO","NM_URNA_CANDIDATO"],
        'dim_turno':["NR_TURNO"],
        'dim_tp_agrangencia':["TP_ABRANGENCIA"],
        'dim_zona':["NR_ZONA"],
        'dim_situacao_candidatura':["DS_SITUACAO_CANDIDATURA"],
        'dim_coligacao':["NM_COLIGACAO", "DS_COMPOSICAO_COLIGACAO"],
        'dim_voto_transito':["ST_VOTO_EM_TRANSITO"],
        'dim_destinacao_votos':["NM_TIPO_DESTINACAO_VOTOS"],
        'dim_totalizacao_candidato':["DS_SIT_TOT_TURNO"]
    },   
)

KeyError: 'DS_SIT_TOT_TURNO'

In [12]:
# Exemplo de como acessar as tabelas:
tabela_fato_nome, df_fato = star_schema[0]

# Visualize o conteúdo da tabela fato
print(f"Conteúdo da Tabela Fato ({tabela_fato_nome}):")
df_fato.show(truncate=False)

# Exiba o esquema da tabela fato
print("Esquema da Tabela Fato:")
df_fato.printSchema()

Conteúdo da Tabela Fato (tabela_fato):
+-------------------------+-----------------+----------------+------------+-----------------+--------------+----------------+------------+---------------------+-----------+---------------------------+----------------+--------------------+-----------------------+----------------------------+
|QT_VOTOS_NOMINAIS_VALIDOS|QT_VOTOS_NOMINAIS|sk_dim_municipio|sk_dim_cargo|sk_dim_ds_eleicao|sk_dim_partido|sk_dim_candidato|sk_dim_turno|sk_dim_tp_agrangencia|sk_dim_zona|sk_dim_situacao_candidatura|sk_dim_coligacao|sk_dim_voto_transito|sk_dim_destinacao_votos|sk_dim_totalizacao_candidato|
+-------------------------+-----------------+----------------+------------+-----------------+--------------+----------------+------------+---------------------+-----------+---------------------------+----------------+--------------------+-----------------------+----------------------------+
|0                        |0                |25769804751     |5           |1         

In [13]:
# Exemplo de como acessar as tabelas de dimensão:
df_dimensao1_nome, df_dimensao1 = star_schema[1]

# Visualize o conteúdo da dimensão 1
print(f"Conteúdo da Dimensão 1 ({df_dimensao1_nome}):")
df_dimensao1.show(truncate=False)

# Exiba o esquema da dimensão 1
print("Esquema da Dimensão 1:")
df_dimensao1.printSchema()

Conteúdo da Dimensão 1 (dim_municipio):
+-----+---------------------+----------------+
|SG_UF|NM_MUNICIPIO         |sk_dim_municipio|
+-----+---------------------+----------------+
|SP   |SÃO PAULO            |0               |
|MG   |DIVISA ALEGRE        |1               |
|MG   |BRAÚNAS              |2               |
|SP   |IGARAPAVA            |3               |
|SP   |BOFETE               |4               |
|SP   |TAUBATÉ              |5               |
|MG   |JACUÍ                |6               |
|PR   |RIO BRANCO DO IVAÍ   |7               |
|MG   |DELTA                |8               |
|MT   |PORTO ALEGRE DO NORTE|9               |
|MG   |DORES DO INDAIÁ      |10              |
|SE   |RIACHÃO DO DANTAS    |11              |
|SP   |IRAPUÃ               |12              |
|PI   |AVELINO LOPES        |13              |
|MA   |SÃO BERNARDO         |14              |
|SC   |BENEDITO NOVO        |15              |
|PA   |MOJU                 |16              |
|BA   |JUSSARI      

In [14]:
# Exemplo de como acessar as tabelas de dimensão:
df_dimensao1_nome, df_dimensao1 = star_schema[2]

# Visualize o conteúdo da dimensão 1
print(f"Conteúdo da Dimensão 1 ({df_dimensao1_nome}):")
df_dimensao1.show(truncate=False)

# Exiba o esquema da dimensão 1
print("Esquema da Dimensão 1:")
df_dimensao1.printSchema()

Conteúdo da Dimensão 1 (dim_cargo):
+---------------------+------------+
|DS_CARGO             |sk_dim_cargo|
+---------------------+------------+
|Deputado Distrital   |0           |
|Governador           |1           |
|Senador              |2           |
|Conselheiro Distrital|3           |
|Presidente           |4           |
|Deputado Estadual    |5           |
|Deputado Federal     |6           |
+---------------------+------------+

Esquema da Dimensão 1:
root
 |-- DS_CARGO: string (nullable = true)
 |-- sk_dim_cargo: long (nullable = false)



In [24]:
# Exemplo de como acessar as tabelas de dimensão:
df_dimensao1_nome, df_dimensao1 = star_schema[4]

# Visualize o conteúdo da dimensão 1
print(f"Conteúdo da Dimensão 1 ({df_dimensao1_nome}):")
df_dimensao1.show(truncate=False)

# Exiba o esquema da dimensão 1
print("Esquema da Dimensão 1:")
df_dimensao1.printSchema()

Conteúdo da Dimensão 1 (dim_partido):
+------------+----------------------------------------------+----------+--------------+
|SG_PARTIDO  |NM_PARTIDO                                    |NR_PARTIDO|sk_dim_partido|
+------------+----------------------------------------------+----------+--------------+
|PSDB        |Partido da Social Democracia Brasileira       |45        |0             |
|AVANTE      |AVANTE                                        |70        |1             |
|AGIR        |AGIR                                          |36        |2             |
|PL          |Partido Liberal                               |22        |3             |
|PDT         |Partido Democrático Trabalhista               |12        |4             |
|PT          |Partido dos Trabalhadores                     |13        |5             |
|PATRIOTA    |Patriota                                      |51        |6             |
|PP          |PROGRESSISTAS                                 |11        |7         

In [13]:
hostname_or_ip = "localhost"
port = "5439"
db = "metabase"
user = "metabase"
password = "metabase"

db_url = "jdbc:postgresql://" + hostname_or_ip + ":" + port + "/" + db

properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}


In [14]:
for item in star_schema:
    table_name,dataframe = item
    print(f"Writing {table_name} to DW")
    dataframe.write.jdbc(url=db_url, table="votacao_candidato."+str(table_name), mode="overwrite", properties=properties)

Writing tabela_fato to DW
Writing dim_municipio to DW
Writing dim_cargo to DW
Writing dim_ds_eleicao to DW
Writing dim_partido to DW
Writing dim_candidato to DW
Writing dim_turno to DW
Writing dim_tp_agrangencia to DW
Writing dim_zona to DW
Writing dim_situacao_candidatura to DW
Writing dim_coligacao to DW
Writing dim_voto_transito to DW
Writing dim_destinacao_votos to DW
Writing dim_totalizacao_candidato to DW


### Cleaning up

In [ ]:
# Stopping spark session
spark_session.stop()

# Cleaning up files 
# Delete the directory and all its contents
# import shutil

# shutil.rmtree(workdir+'extracted/')